# 年轻人的第一个爬虫
该怎么描述爬虫呢，你可以叫它 `crawler`，也可以叫它 `spider`。
由于互联网的信息太多了，完全靠人去收集显然不太现实，就有了自动收集的相关技术。
由于这个采集过程就像一个蜘蛛在网上漫游，所以才被称为爬虫。

在本章节，我们会学习基础的 HTTP 相关知识，Robots 协议，以及使用 Request+BeautifulSoup/lxml 来编写一个简单的爬虫。

## HTTP

详细的介绍 HTTP 并不是我们教程的目的，我们只需要明白，HTTP 由请求类型，请求头，空行，请求体组成。我们分别介绍一下这些内容。

### HTTP 头

通过 HTTP 首部信息，客户端或者服务器端就能了解到这次报文到底具有哪些属性，报文发送端有哪些喜好等等。

HTTP 首部有以下几种分类：
1. 通用首部：请求报文和响应报文都支持，换句话说，就是既可以出现在请求报文中，也可以出现在响应报文中。
2. 请求首部：出现在请求报文第一行（请求行）的后面，为请求报文添加一些附加信息。
3. 响应首部：出现在响应报文第一行的后面，提供了一些关于响应报文的一些信息。
4. 实体首部：我们都知道，请求报文和响应报文都包含实体报文，实体首部就是用来描述实体报文的一些属性。
5. 扩展头部：HTTP 规范中没有定义的首部。

对于我们写爬虫程序的，大部分时间我们需要模拟 HTTP 请求发送到服务端，所以在分析 HTTP 请求的时候，我们需要更加关注一下 HTTP 请求头 （Request Header） ，它用来向请求报文或者响应报文中添加一些附加信息。

|部分 Request Header|解释|
|-|-|
|Accept|指定客户端能够接收的内容类型|
|Accept-Charset|浏览器可以接受的字符编码集|
|Accept-Encoding|指定浏览器可以支持的web服务器返回内容压缩编码类型|
|Cache-Control|指定请求和响应遵循的缓存机制|
|Cookie|HTTP 请求发送时，会把保存在该请求域名下的所有 cookie 值一起发送给服务器|
|Connection|表示是否需要持久连接|
|Content-Type|请求的与实体内容类型|
|Content-Length|请求的内容长度|
|Referer|请求的来源 url |
|User-Agent|User-Agent 的内容包含发出请求的用户信息|	

我们需要特别关注几个比较重要的头字段：

- Accept
  最常见为 `Accept */*` ，指接受所有类型除此之外还有 `text/plain`，`text/html`，分别指接受纯文本和 html 格式。
- Content-Type
  指定 HTTP 内容部分的类型，它有以下几种常见的值：
  - `application/x-www-form-urlencoded`
    数据被编码为 名称/值 的对。这是标准的编码格式。该编码方式把会把数据转换成一个字串的形式（name1=value1&name2=value2...），jquery 的 ajax 方法默认使用的就是该格式。
  - `multipart/form-data`
    传输文件经常时会使用。它会将表单的数据以名称为单元，用分隔符分开，分别标注类型和存储数值。
  - `text/plain`
    数据以纯文本形式进行编码。
  - `application/json`
    数据以 json 的格式进行编码。
- Connection
  常见值为 `keep-alive`，表示之前已经进行过握手，可以直接进行HTTP正文传输。而 `close` 则表示结束，接下来没有东西再传输了。在 HTTP/1.0 的时代，一个 HTTP 请求就要 TCP 三次握手和四次挥手，当一个网页中包含大量的图片或者其它外部资源时就会造成了网络资源的浪费。到了 HTTP/1.1 的时候，通过请求头的 Connection 字段用来申明，长时间保持连接，即使有大量的图片和外部资源，也只用进行一次握手，这样就大大的减少了传输量了。

## A1：模拟 HTTP 交互

这是一个练习，帮助你更好的理解 HTTP 请求，只看不做等于不会。在本练习中，我们会使用 Socket 来模拟服务器和客户端，当然它非常的简陋，不过来完成我们的 HTTP 请求和响应足够了。

In [3]:
"""BaseHTTPServer that implements the Python WSGI protocol (PEP 3333)

This is both an example of how WSGI can be implemented, and a basis for running
simple web applications on a local machine, such as might be done when testing
or debugging an application.  It has not been reviewed for security issues,
however, and we strongly recommend that you use a "real" web server for
production use.

For example usage, see the 'if __name__=="__main__"' block at the end of the
module.  See also the BaseHTTPServer module docs for other API information.
"""

from http.server import BaseHTTPRequestHandler, HTTPServer
import sys
import urllib.parse
from wsgiref.handlers import SimpleHandler
from platform import python_implementation

__version__ = "0.2"


server_version = "WSGIServer/" + __version__
sys_version = python_implementation() + "/" + sys.version.split()[0]
software_version = server_version + ' ' + sys_version


class ServerHandler(SimpleHandler):

    server_software = software_version

    def close(self):
        try:
            self.request_handler.log_request(
                self.status.split(' ',1)[0], self.bytes_sent
            )
        finally:
            SimpleHandler.close(self)



class WSGIServer(HTTPServer):

    """BaseHTTPServer that implements the Python WSGI protocol"""

    application = None

    def server_bind(self):
        """Override server_bind to store the server name."""
        HTTPServer.server_bind(self)
        self.setup_environ()

    def setup_environ(self):
        # Set up base environment
        env = self.base_environ = {}
        env['SERVER_NAME'] = self.server_name
        env['GATEWAY_INTERFACE'] = 'CGI/1.1'
        env['SERVER_PORT'] = str(self.server_port)
        env['REMOTE_HOST']=''
        env['CONTENT_LENGTH']=''
        env['SCRIPT_NAME'] = ''

    def get_app(self):
        return self.application

    def set_app(self,application):
        self.application = application



class WSGIRequestHandler(BaseHTTPRequestHandler):

    server_version = "WSGIServer/" + __version__

    def get_environ(self):
        env = self.server.base_environ.copy()
        env['SERVER_PROTOCOL'] = self.request_version
        env['SERVER_SOFTWARE'] = self.server_version
        env['REQUEST_METHOD'] = self.command
        if '?' in self.path:
            path,query = self.path.split('?',1)
        else:
            path,query = self.path,''

        env['PATH_INFO'] = urllib.parse.unquote(path, 'iso-8859-1')
        env['QUERY_STRING'] = query

        host = self.address_string()
        if host != self.client_address[0]:
            env['REMOTE_HOST'] = host
        env['REMOTE_ADDR'] = self.client_address[0]

        if self.headers.get('content-type') is None:
            env['CONTENT_TYPE'] = self.headers.get_content_type()
        else:
            env['CONTENT_TYPE'] = self.headers['content-type']

        length = self.headers.get('content-length')
        if length:
            env['CONTENT_LENGTH'] = length

        for k, v in self.headers.items():
            k=k.replace('-','_').upper(); v=v.strip()
            if k in env:
                continue                    # skip content length, type,etc.
            if 'HTTP_'+k in env:
                env['HTTP_'+k] += ','+v     # comma-separate multiple headers
            else:
                env['HTTP_'+k] = v
        return env

    def get_stderr(self):
        return sys.stderr

    def handle(self):
        """Handle a single HTTP request"""

        self.raw_requestline = self.rfile.readline(65537)
        if len(self.raw_requestline) > 65536:
            self.requestline = ''
            self.request_version = ''
            self.command = ''
            self.send_error(414)
            return

        if not self.parse_request(): # An error code has been sent, just exit
            return

        handler = ServerHandler(
            self.rfile, self.wfile, self.get_stderr(), self.get_environ(),
            multithread=False,
        )
        handler.request_handler = self      # backpointer for logging
                envs = handler.base_env
        
        if self.headers['content-type'] == "application/x-www-form-urlencoded":
            if self.headers.get('content-length'):
                content = self.rfile.read(int(self.headers['content-length'])).decode("utf-8")
                content_list = content.split("&")
                for one in content_list:
                    kv = one.split("=")
                    envs[kv[0]] = kv[1]
        handler.run(self.server.get_app())


def demo_app(environ, start_response):
    from io import StringIO
    stdout = StringIO()
    header_keys = ['SERVER_PORT', 'REMOTE_HOST',
                   'CONTENT_LENGTH', 'SERVER_PROTOCOL', 'REQUEST_METHOD', 'PATH_INFO',
                   'QUERY_STRING', 'REMOTE_ADDR', 'CONTENT_TYPE', 'HTTP_HOST', 'HTTP_CONNECTION',
                   'HTTP_USER_AGENT', 'HTTP_ACCEPT_LANGUAGE', 'HTTP_ACCEPT']
    print("Your request get response successfully", file=stdout)
    print("Your request content: ")
    if "username" in environ:
        print("username", '=', environ["username"], file=stdout)
    if "password" in environ:
        print("password", '=', environ["password"], file=stdout)
    print(file=stdout)
    for k in header_keys:
        if k in environ:
            print(k, '=', repr(environ[k]), file=stdout)
        else:
            print(k, '=', file=stdout)
    start_response("200 OK", [('Content-Type', 'text/plain; charset=utf-8')])
    return [stdout.getvalue().encode("utf-8")]


def make_server(
    host, port, app, server_class=WSGIServer, handler_class=WSGIRequestHandler
):
    """Create a new WSGI server listening on `host` and `port` for `app`"""
    server = server_class((host, port), handler_class)
    server.set_app(app)
    return server


In [ ]:
from lecture1.code.client import Client

client = Client()


def run_get_test():
    client.do_get("127.0.0.1", 8001)

def run_post_test():
    client.do_post("127.0.0.1", 8001)


def run_server():
    with make_server('127.0.0.1', 8001, demo_app) as httpd:
        sa = httpd.socket.getsockname()
        print("Serving HTTP on", sa[0], "port", sa[1], "...")
        httpd.handle_request()  # serve one request, then exit

def test_with_timeout(s_fn, client_t, timeout=10):
    lock = threading.Lock()
    result = False

    def run_callback():
        fn()
        with lock:
            result = True


    server_t = threading.Thread(target=run_callback)
    server_t.daemon = True
    server_t.start()
    time.sleep(1)
    client_t.start()
    client_t.join(timeout)
    with lock:
        return result


Serving HTTP on 127.0.0.1 port 8001 ...


In [ ]:
client_t = threading.Thread(target=run_get_test)
test_with_timeout(run_server, client_t)

In [ ]:
client_t = threading.Thread(target=run_post_test)
test_with_timeout(run_server, client_t)

## Robots 协议

互联网并非法外之地，~~“爬虫写得好，牢饭吃到饱”~~

<table align=left>
<td><img src="./asset/maomao.png" width="200"></td>
<td><img src="./asset/maomao2.png" width="200"></td>
</table>

In [ ]:
编写爬虫程序，我们要遵守相关的规定，最重要的就是 Robots协议。大多数网站都会定义 `robots.txt`文件，这是一个君子协议。它规定了爬虫不允许访问那些目录下的内容。

如淘宝网的 Robots.txt 文件

```
User-agent: Baiduspider
Disallow: /
```
它就表示不允许百度对其任何内容的爬取访问。

## DOM 树和 HTML 解析

DOM 是 Document Object Model（文档对象模型）的缩写。当你的服务器把 HTML 网页发送给用户时，用户的浏览器会解析你的 HTML 代码，生成 DOM 树，CSS 则会生成 CSS 规则树。有了这两个树，你的浏览器就可以渲染(生成整个页面)了！所有最终呈现的页面都是由我们的浏览器渲染生成的，这也是为什么我们在检查页面时(f12)，对内容的修改会直接反应到页面上。

除了直接使用正则对 HTML 的进行字符串的解析，还有一种方法就是类似于浏览器构造整个 DOM 树，先将 HTML 文本构建成一个树形的结构，这样我们可以轻松定位到对应的标签位置和文本位置。

Python 解析 HTML 常用的库有 `lxml` 和 `BeautifulSoup` 它们就是基于树的方法。

## A2：利用 Beautiful Soup 解析 HTML

这是一个练习，帮助你掌握如何使用 BeautifulSoup 解析 HTML。在本练习中，我们会提供一段以字符串表示的 HTML 文本，你需要利用 `BeautifulSoup` 等工具来解析该字符串，提取其中对应的文字。你可以参考[官方文档](https://beautifulsoup.readthedocs.io/zh_CN/v4.4.0/) 了解更多的函数使用。

In [ ]:
# 安装 bs4 库
%pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
html_doc = """
<html><head><title>The Dormouse's story</title><link rel="shortcut icon" href="/favicon.ico" type="image/x-icon"></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

soup = BeautifulSoup(html_doc, 'html.parser')

# q1 请你获取所有 <a> 标签的超链接地址以及其对应链接附着的 HTML 文字
print()
# q2 请你获取该 HTML 的标签标题和标签 Logo 的地址
print()
# q3 请你获取全部的文字内容，注意不要包含 HTML 标记符号 !提示：文字内容一般以段落标签 <p> 来标识
print()
# q4 请你使用至少三种方式（a 标签，class 属性，DOM 树结构）找到该 HTML 的第一个超链接的地址
print()

## 脱离浏览器的请求：requests 库

还记得第一个作业吗，我们使用两端代码来分别模拟客户端和服务器，将 HTTP 请求以原生的二进制分段一点点发出去，并以同样方式一点点接收 HTTP 响应。如果随便写个爬虫都要重复这些步骤的话，也太麻烦了，所以我们就有了更高级的封装代码。Python 内置一个 `Urllib` 库可以使用，但相较于它，大家更喜欢轻快便捷的 `requests` 库。

它可以简单的一行代码完成各种请求，常见的 `GET` 和 `POST` 的写法如下 

```python
r = requests.get('https://api.github.com/events')
r = requests.post('https://somewebsite.org/post', data = {'key':'value'})
```

完成 HTTP 请求后，我们通常使用 `r.text` 来获得返回的文本，之后如何解析该文本，就是我们爬虫该做的工作了。

## A3：年轻人的第一个爬虫

点题，这是一个全面的练习，需要你掌握一定的 DOM 分析能力，以及熟练运用 F12 观察 HTML 文本，加上一点点思考。

本次需要你独立完成这样一个程序：对于用户的输入内容，请你爬取 **百度搜索的第一个显示标题和对应的网址（非广告内容）**。推荐使用 `requests.get` 和 `BeautifulSoup` 来完成。

在做的时候请思考以下问题：
- HTTP Header 是否需要添加，如果需要，请说出你都加入了那些内容，并都发挥着什么作用。
- 使用 HTTPS 和 HTTP，有什么区别

如果结果正确，你输入 Python 的结果应该返回 `Welcome to Python.org` 和 `https://www.python.org/`

In [ ]:
# 安装 requests 库
%pip install requests


keyword = input()
r = requests.get("url with keyword")
print(r.text)
# 请开始你的解析